In [ ]:
import os

def build_depth_map(super_path):
    depth_map_path = {}

    # Walk through the directory tree
    for root, _, files in os.walk(super_path):
        for filename in files:
            if filename.endswith('.png'):
                full_path = os.path.join(root, filename)
                # Remove super_path from the full path to get the relative path
                relative_path = os.path.relpath(full_path, super_path)
                # Split the relative path into parts (folders)
                parts = relative_path.split(os.sep)
                parts[-1]=parts[-1][len('result_merged_depth_'):-len('.png')]
                
                # Initialize nested dictionaries as needed
                current_dict = depth_map_path
                for part in parts[:-1]:  # Iterate over all parts except the last one (filename)
                    current_dict = current_dict.setdefault(part, {})
                
                # Assign the full path to the deepest nested dictionary
                current_dict[parts[-1]] = full_path

    return depth_map_path

def build_depth_map_confidence(super_path):
    depth_map_path = {}

    # Walk through the directory tree
    for root, _, files in os.walk(super_path):
        for filename in files:
            if filename.endswith('.exr'):
                full_path = os.path.join(root, filename)
                # Remove super_path from the full path to get the relative path
                relative_path = os.path.relpath(full_path, super_path)
                # Split the relative path into parts (folders)
                parts = relative_path.split(os.sep)
                parts[-1]=parts[-1][len('result_merged_conf_'):-len('.exr')]
                
                # Initialize nested dictionaries as needed
                current_dict = depth_map_path
                for part in parts[:-1]:  # Iterate over all parts except the last one (filename)
                    current_dict = current_dict.setdefault(part, {})
                
                # Assign the full path to the deepest nested dictionary
                current_dict[parts[-1]] = full_path

    return depth_map_path



In [ ]:
import os

def find_train_folders(directory):
    train_folders = []
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            if dir.startswith('train'):
                train_folders.append(os.path.join(root, dir))
    return train_folders

train_path=find_train_folders('/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/')

depth_map_path={}
depth_map_path_temp={}

for path in train_path:
    depth_map_path_temp=build_depth_map(path+"/merged_depth")
    depth_map_path.update(depth_map_path_temp)

depth_map_confidence_path={}
depth_map_confidence_path_temp={}

for path in train_path:
    depth_map_confidence_path_temp=build_depth_map_confidence(path+"/merged_conf")
    depth_map_confidence_path.update(depth_map_confidence_path_temp)




import pprint
pprint.pprint(depth_map_path)
pprint.pprint(depth_map_confidence_path)


In [ ]:
import numpy as np
import random

def generate_random_patches(size=(504,378)):
    image=np.zeros(size)
    patches=[]
    for _ in (range(100)):
        x=random.randint(16, 487)
        y=random.randint(16, 361)

        #print(x,y)
        
        breaker=False

        for i in range(x-26,x+26):
            for j in range(y-26,y+26):
                if i<0 or i>503 or j<0 or j>377 :
                    continue
                if image[i][j]==1:
                    breaker=True
                    break
            if breaker:
                break

        if breaker:
            continue
        
        x=x-16
        y=y-16

        for i in range(x,x+33):
            for j in range(y,y+33):
                image[i][j]=1

        patches.append((x,y))


    return patches

In [ ]:
import numpy as np
import cv2
import OpenEXR
import Imath

def load_exr(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    header = exr_file.header()
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1
    pt = Imath.PixelType(Imath.PixelType.FLOAT)
    r = np.frombuffer(exr_file.channel('R', pt), dtype=np.float32)
    r.shape = (height, width)
    return r

def predict_patches(depth_confidence_path):
    depth_map_confidence = load_exr(depth_confidence_path)

    all_patches=generate_random_patches()

    confidence_patch_blocks=[depth_map_confidence[x:x+32,y:y+32] for (x,y) in all_patches]
    median_confidence_patch_block=[np.median(patch.flatten()) for patch in confidence_patch_blocks]

    patch_indices=[i for i, value in enumerate(median_confidence_patch_block) if value >= 0.98]

    if len(patch_indices)>4:
        patch_indices = sorted(range(len(median_confidence_patch_block)), key=lambda i: median_confidence_patch_block[i], reverse=True)[:4]

    patches=[all_patches[patch_index] for patch_index in patch_indices]

    return patches

def predict_focal_length(depth_map_path,patch):
    depth_map = cv2.imread(depth_map_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)

    depth_values=depth_map[patch[0]:patch[0]+32,patch[1]:patch[1]+32]
    depth_values=depth_values.flatten()

    # Define max and min values
    max_depth = 100.0
    min_depth = 0.2

    depth_map_in_meters = (max_depth * min_depth) / (max_depth - (max_depth - min_depth) * (depth_values / 255.0))

    # Compute the median value in the entire depth map
    median_depth = np.median(depth_map_in_meters)

    final_focus=median_depth*1000

    return final_focus

slice_focal_length=[3910.92,2289.27,1508.71,1185.83,935.91,801.09,700.37,605.39,546.23,486.87,447.99,407.40,379.91,350.41,329.95,307.54,291.72,274.13,261.53,247.35,237.08,225.41,216.88,207.10,198.18,191.60,183.96,178.29,171.69,165.57,160.99,155.61,150.59,146.81,142.35,138.98,134.99,131.23,127.69,124.99,121.77,118.73,116.40,113.63,110.99,108.47,106.54,104.23,102.01]

def find_closest(value, num_list):
    closest_value = min(num_list, key=lambda x: abs(x - value))
    return closest_value

def predict_slice(depth_map_path,depth_confidence_path):
    truth=[]

    patches=predict_patches(depth_confidence_path)
    
    for patch in patches:
        predicted_focus=predict_focal_length(depth_map_path,patch)
        closest_value = find_closest(predicted_focus, slice_focal_length)
        true_slice=slice_focal_length.index(closest_value)
        true_slice=true_slice-1 if true_slice !=0 else true_slice
        truth.append((patch[0],patch[1],true_slice))

    return truth

In [ ]:
from tqdm import tqdm
import copy

ground_truth=copy.deepcopy(depth_map_path)

for image_type in tqdm(ground_truth):
    for pos in ground_truth[image_type]:
        ground_truth[image_type][pos]=predict_slice(depth_map_path[image_type][pos],depth_map_confidence_path[image_type][pos])

import pprint
pprint.pprint(ground_truth)

In [ ]:
patch_count=[]

for key1 in ground_truth:
    for key2 in ground_truth[key1]:
        patch_count.append(len(ground_truth[key1][key2]))

print(min(patch_count)," ",np.mean(patch_count)," ",max(patch_count))

In [ ]:
import os

def generate_image_paths(super_path):
    # Initialize the dictionary
    image_path = {}

    # Traverse the directory structure
    for root, dirs, files in os.walk(super_path):
        for file in files:
            if file.endswith('.png'):
                # Get the relative path from super_path
                relative_path = os.path.relpath(root, super_path)
                # Split the relative path to get <string> and <integer>
                string_part, integer_part = os.path.split(relative_path)
                integer_part = int(integer_part)
                
                # Extract the position from the filename
                file_name_parts = file.split('_')
                position = file_name_parts[-1].split('.')[0]  # Extract 'bottom', 'top', etc.
                
                # Construct the full file path
                full_file_path = os.path.join(root, file)
                
                # Populate the dictionary
                if string_part not in image_path:
                    image_path[string_part] = {}
                if integer_part not in image_path[string_part]:
                    image_path[string_part][integer_part] = {}
                
                image_path[string_part][integer_part][position] = full_file_path

    return image_path



In [ ]:

left_image_path = {}
left_image_path_temp = {}
right_image_path = {}
right_image_path_temp = {}


def find_train_folders(directory):
    train_folders = []
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            if dir.startswith('train'):
                train_folders.append(os.path.join(root, dir))
    return train_folders


train_path = find_train_folders(
    '/mnt/Velocity Vault/Personal/Projects/Python/Autofocus/Train/')


for path in train_path:
    left_image_path_temp = generate_image_paths(path+'/raw_up_left_pd')
    left_image_path.update(left_image_path_temp)
    right_image_path_temp = generate_image_paths(path+'/raw_up_right_pd')
    right_image_path.update(right_image_path_temp)




In [ ]:
def reorder_image_path(image_path):
    ip = {}

    for key1, level2_dict in image_path.items():
        for key2, level3_dict in level2_dict.items():
            for key3, value in level3_dict.items():
                if key1 not in ip:
                    ip[key1] = {}
                if key3 not in ip[key1]:
                    ip[key1][key3] = []
                ip[key1][key3].append(value)

    return ip

left_image_path=reorder_image_path(left_image_path)
right_image_path=reorder_image_path(right_image_path)


import pprint

pprint.pprint(left_image_path)
pprint.pprint(right_image_path)
print(len(left_image_path))
print(len(right_image_path))



In [ ]:
def combine_image_paths(left_image_path,right_image_path):
    image_paths=copy.deepcopy(left_image_path)
    for key1 in image_paths:
        for key2 in image_paths[key1]:
            image_paths[key1][key2]=left_image_path[key1][key2]+right_image_path[key1][key2]
    return image_paths

image_paths=combine_image_paths(left_image_path,right_image_path)

pprint.pprint(image_paths)
print(len(image_paths))
print(type(image_paths['apt1_0']['center']))
print(len(image_paths['apt1_0']['center']))

In [ ]:
def collapse_last_dimension(arr):
    # Ensure the array has the correct shape
    assert arr.shape[-1] == 3, "Input array must have shape AxBx3"
    
    # Extract the first two dimensions (A and B)
    collapsed_arr = arr[..., 0]
    
    return collapsed_arr

def combine_last_dimension(arrays):
    A = arrays[0].shape[0]  # Assuming all arrays have the same shape AxA
    s = len(arrays)
    
    # Initialize the resulting array with lists
    list_array = np.empty((A, A, s), dtype=object)
    
    # Fill the array with corresponding elements from each input array
    for k, array in enumerate(arrays):
        list_array[..., k] = array
    
    return list_array


In [ ]:
from tqdm import tqdm

def build_dataset_labels(image_paths,ground_truth):
    dataset=[]
    labels=[]

    for image_type in tqdm(ground_truth):
        for pos in ground_truth[image_type]:
            temp=ground_truth[image_type][pos]
            patches=[(x,y) for (x,y,_) in temp]
            truth=[z for (_,_,z) in temp]

            for patch in patches:
                image_path=image_paths[image_type][pos]
                images=[]
                for path in image_path:
                    x=patch[0]*4
                    y=patch[1]*4
                    image=cv2.imread(path)
                    image=collapse_last_dimension(image)
                    image=image[x:x+128,y:y+128]
                    images.append(image)
                image_set=combine_last_dimension(images)
                dataset.append(image_set)
                labels.append(truth)

    return dataset,labels

dataset,labels=build_dataset_labels(image_paths,ground_truth)

pprint.pprint(dataset)
pprint.pprint(labels)
print(len(dataset))
print(len(labels))

In [ ]:
print(dataset[0].shape)

def check_shapes(lst):
    target_shape = (128, 128, 98)
    for element in lst:
        if not isinstance(element, np.ndarray) or element.shape != target_shape:
            return False
    return True

print(check_shapes(dataset))

In [ ]:
import pickle

def save_list_to_file(dataset, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dataset, file)

def load_list_from_file(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)
    

In [ ]:

save_list_to_file(dataset,'dataset_cache.pkl')

In [ ]:

#save_list_to_file(dataset,'dataset_cache.pkl')

loaded_dataset=load_list_from_file('dataset_cache.pkl')

pprint.pprint(loaded_dataset)
print(len(loaded_dataset))
print(check_shapes(loaded_dataset))

#print(dataset==loaded_dataset)

In [ ]:

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Step 1: Modify MobileNetV2 to accept 128x128x98 input
def create_modified_mobilenetv2(input_shape=(128, 128, 98)):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights=None)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # Assuming the output for the ordinal regression problem is a single value
    outputs = Dense(1)(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

# Step 2: Implement the ordinal regression loss
def ordinal_regression_loss(y_true, y_pred):
    # L2 loss (mean squared error)
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Step 3: Set up the training loop
def train_model(model, train_dataset, steps_per_epoch=20000, batch_size=128, initial_lr=1e-3, beta1=0.5, beta2=0.999):
    optimizer = Adam(learning_rate=initial_lr, beta_1=beta1, beta_2=beta2)
    model.compile(optimizer=optimizer, loss=ordinal_regression_loss)
    
    model.fit(train_dataset, epochs=int(steps_per_epoch // (len(train_dataset) / batch_size)), steps_per_epoch=steps_per_epoch)

# Create the model
input_shape = (128, 128, 98)
model = create_modified_mobilenetv2(input_shape=input_shape)

x_train=dataset.copy()
y_train=labels.copy()

model.summary()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(buffer_size=10000)

train_model(model, train_dataset, steps_per_epoch=59, batch_size=32)
